In [ ]:
#! pip install mlflow boto3 awscli optuna xgboost imbalanced-learn
#! pip install optuna

In [ ]:
import mlflow
# step 1 : Set up the mlflow tracking server 
mlflow.set_tracking_uri("http://ec2-13-62-127-174.eu-north-1.compute.amazonaws.com:5000/")

In [3]:
# Set or create an experiment
mlflow.set_experiment("Exp 5 - ML Algos with HP Tuning")

2025/11/17 21:51:27 INFO mlflow.tracking.fluent: Experiment with name 'Exp 5 - ML Algos with HP Tuning' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://reddit-reccomender-bucket/11', creation_time=1763396487435, experiment_id='11', last_update_time=1763396487435, lifecycle_stage='active', name='Exp 5 - ML Algos with HP Tuning', tags={}>

In [7]:
import optuna
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report,f1_score
from sklearn.feature_extraction.text import CountVectorizer
from xgboost import XGBClassifier
from imblearn.under_sampling import RandomUnderSampler
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


In [6]:
df = pd.read_csv('reddit_preprocessing.csv').dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [8]:
# Step 1: Remap the class labels from [-1, 0, 1] to [2, 0, 1] as xgboost doesnt read -1
df['category'] = df.category.map({-1:2,0:0,1:1})

# step 2:Remove rows where target label asre null
df = df.dropna(subset=['category'])

ngram_range = (1,3) #trigram from previous exp
max_features = 1000 # max_feature from previous experiment

# step 3: train test split before vectorization and resampling
X_train,X_test,y_train,y_test = train_test_split(df['clean_comment'],df['category'],test_size=0.2,random_state=42,stratify=df['category'])

#step 4: vectorization using BOW , fit on training data only
vectorizer = CountVectorizer(ngram_range=ngram_range,max_features=max_features)
X_train_vec= vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# use undersampler method as it came out best in exp4
rus = RandomUnderSampler(random_state=42)
X_train_vec,y_train = rus.fit_resample(X_train_vec,y_train)

# Ensure integer labels
y_train = y_train.astype(int)
y_test = y_test.astype(int)

#Function to log results in MLflow
def log_mlflow(model_name,model,X_train,X_test,y_train,y_test):
    with mlflow.start_run():
        #log the model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_Undersampling_BOW_Trigram")
        mlflow.set_tag("experiment_type", "Algorithm comparision")

        #log algorith name as parameter
        mlflow.log_param("algo_name",model_name)

        #Train model
        model.fit(X_train,y_train)
        y_pred =model.predict(X_test)

        # log accuracy
        accuracy = accuracy_score(y_test,y_pred)
        mlflow.log_metric("accuracy",accuracy)
        f1_macro = f1_score(y_test, y_pred, average="macro")
        mlflow.log_metric("f1_macro",f1_macro)
        f1_weighted = f1_score(y_test, y_pred, average="weighted")
        mlflow.log_metric("f1_weighted",f1_weighted)


        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    if metric != "support":
                        mlflow.log_metric(f"{label}_{metric}", value)

        #Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")

#step 6: Optuna objective function for XG boost
def objective_xgboost(trial):
    n_estimators = trial.suggest_int('n_estimators',50,300)
    learning_rate = trial.suggest_float('learning_rate',1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int('max_depth',3,10)

    model = XGBClassifier(
    n_estimators=n_estimators,
    learning_rate=learning_rate,
    max_depth=max_depth,
    random_state=42,
    eval_metric="logloss",
    objective="multi:softmax",   # FIX
    num_class=3                  # FIX
)
    
    preds = model.fit(X_train_vec, y_train).predict(X_test_vec)
    return f1_score(y_test, preds, average='weighted')

#stage 7 :run otuna for XG boost , log in the best model only 
def run_optuna_experiment():
    study = optuna.create_study(direction= "maximize")
    study.optimize(objective_xgboost,n_trials =30)

    #get the best parameter and log only the best parameter
    best_params = study.best_params
    best_model = XGBClassifier(
    n_estimators=best_params['n_estimators'],
    learning_rate=best_params['learning_rate'],
    max_depth=best_params['max_depth'],
    random_state=42,
    eval_metric="logloss",
    objective="multi:softmax",
    num_class=3
)

        
    # Log the best model with MLflow, passing the algo_name as "xgboost"
    log_mlflow("XGBoost", best_model, X_train_vec, X_test_vec, y_train, y_test)

#run the experiment for XG boost
run_optuna_experiment()







[I 2025-11-18 00:28:49,556] A new study created in memory with name: no-name-3fcbde0a-eef2-4580-9c40-031b61961a09
[I 2025-11-18 00:28:51,084] Trial 0 finished with value: 0.7440771380049607 and parameters: {'n_estimators': 130, 'learning_rate': 0.08992686140335425, 'max_depth': 7}. Best is trial 0 with value: 0.7440771380049607.
[I 2025-11-18 00:28:52,629] Trial 1 finished with value: 0.6114513876364698 and parameters: {'n_estimators': 95, 'learning_rate': 0.008840014805722529, 'max_depth': 7}. Best is trial 0 with value: 0.7440771380049607.
[I 2025-11-18 00:28:53,264] Trial 2 finished with value: 0.47641553065298137 and parameters: {'n_estimators': 98, 'learning_rate': 0.00010015873986564972, 'max_depth': 4}. Best is trial 0 with value: 0.7440771380049607.
[I 2025-11-18 00:28:54,038] Trial 3 finished with value: 0.6759131879712167 and parameters: {'n_estimators': 69, 'learning_rate': 0.06217254759579635, 'max_depth': 6}. Best is trial 0 with value: 0.7440771380049607.
[I 2025-11-18 00

🏃 View run XGBoost_Undersampling_BOW_Trigram at: http://ec2-51-21-223-34.eu-north-1.compute.amazonaws.com:5000/#/experiments/11/runs/84eccf773a75443e88e4d80d656a3681
🧪 View experiment at: http://ec2-51-21-223-34.eu-north-1.compute.amazonaws.com:5000/#/experiments/11
